In [1]:
import pickle, random, copy, platform, os
import numpy as np
import tensorflow as tf
import email_sending
from PIL import Image, ImageDraw

In [2]:
def vessel_img():
    with open('vessel', 'rb') as vessel:
        data = pickle.load(vessel)
        band1 = data[0]
        band2 = data[1]

    return band1, band2

In [3]:
#band1
band1 = vessel_img()[0]
band2 = vessel_img()[1]

In [4]:
X1 = band1.reshape(-1, 5625)
X2 = band2.reshape(-1, 5625)

In [ ]:
'''X1 = np.array([X1[0] for i in range(len(X1))])
X2 = np.array([X2[0] for i in range(len(X1))])'''

In [5]:
# GAN Modeling

import matplotlib.pyplot as plt

total_epoch = 10000
batch_size = 37 #851=37*23
learning_rate = 0.00002
# layer option
n_noise = 1024 # the size of noise 
n_input = 75*75
n_hidden =2024

In [6]:
# X is the input, we do not use the labels Y of images
X = tf.placeholder(tf.float32, [None, n_input])
# Z is the noise 
Z = tf.placeholder(tf.float32, [None, n_noise])

In [7]:
g_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], mean=0, stddev=0.01, dtype=tf.float32))
g_b1 = tf.Variable(tf.random_normal([n_hidden],stddev=0.01))
g_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], mean=0, stddev=0.01, dtype=tf.float32))
g_b2 = tf.Variable(tf.random_normal([n_input],stddev=0.01))

In [8]:
def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z, g_W1) + g_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, g_W2) + g_b2)
    Gen_IMG = tf.reshape(output, [-1, 75, 75, 1])
    return Gen_IMG

In [9]:
d_W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
d_W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
d_W3 = tf.get_variable("d_W3", shape=[64*19*19, n_noise], initializer=tf.contrib.layers.xavier_initializer())
d_b1 = tf.Variable(tf.random_normal([n_noise]))
d_W4 = tf.get_variable("d_W4", shape=[n_noise, 1], initializer=tf.contrib.layers.xavier_initializer())
d_b2 = tf.Variable(tf.random_normal([1]))

In [10]:
def discriminator(inputs):
    X_img = tf.reshape(inputs, [-1, 75, 75, 1]) 
    
    #Layer1
    L1 = tf.nn.conv2d(X_img, d_W1, strides=[1, 1, 1, 1], padding='SAME')
    L1 = tf.nn.relu(L1)
    L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')
    #Layer2
    L2 = tf.nn.conv2d(L1, d_W2, strides=[1, 1, 1, 1], padding='SAME')
    L2 = tf.nn.relu(L2)
    L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')
    L2_flat = tf.reshape(L2, [-1, 64 * 19 * 19 ])
    
   # L3 FC 64*19*19 inputs -> 1024 outputs
    L3 = tf.nn.relu(tf.matmul(L2_flat, d_W3) + d_b1)
    #Final FC 1024 inputs -> 1 outputs
    logits = tf.matmul(L3, d_W4) + d_b2
    output = tf.nn.sigmoid(logits)
    return output

In [11]:
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [12]:
# Generate the random image
G = generator(Z)
# Discriminate the generated image
D_generate = discriminator(G)
# Discriminate the real image
D_real = discriminator(X)

In [13]:
loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                        logits=D_real, labels=tf.ones_like(D_real)))

In [14]:
loss_D_generate = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
    logits=D_generate, labels=tf.zeros_like(D_generate)))

In [15]:
loss_D = loss_D_real + loss_D_generate

In [16]:
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_generate, labels=tf.ones_like(D_generate)))

In [17]:
d_var_list = [d_W1, d_b1, d_W2, d_b2, d_W3, d_W4]
g_var_list = [g_W1, g_b1, g_W2, g_b2]

In [18]:
train_D = tf.train.AdamOptimizer(learning_rate).minimize(loss_D,var_list=d_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(loss_G,var_list=g_var_list)

In [19]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(851/batch_size)
loss_val_D, loss_val_G = 0, 0

In [ ]:
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs= X1[i*batch_size:(i+1)*batch_size]
        noise = get_noise(batch_size, n_noise)

        # learn the discriminator and generator.
        _, loss_val_D = sess.run([train_D, loss_D],
                                 feed_dict={X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G],
                                 feed_dict={Z: noise})

    print('Epoch:', '%04d' % epoch,
          'D loss: {:.4}'.format(loss_val_D),
          'G loss: {:.4}'.format(loss_val_G))
    
    #########
    # save the images which is generated
    ######
    if epoch == 0 or (epoch + 1) % 5 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})

        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))

        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (75, 75)), cmap=plt.cm.bone, interpolation='nearest')

        plt.savefig('samples/{}.png'.format(str(epoch).zfill(4)), bbox_inches='tight')
        plt.close(fig)
        print(samples[0])
print('최적화 완료!')

Epoch: 0000 D loss: 1.516 G loss: 0.3584
[[[ 0.50121456]
  [ 0.49474573]
  [ 0.45559821]
  ..., 
  [ 0.46660626]
  [ 0.47807986]
  [ 0.4920575 ]]

 [[ 0.48666385]
  [ 0.47334656]
  [ 0.48912436]
  ..., 
  [ 0.47692072]
  [ 0.4755801 ]
  [ 0.50712115]]

 [[ 0.52500838]
  [ 0.47780699]
  [ 0.43510625]
  ..., 
  [ 0.48568064]
  [ 0.52183127]
  [ 0.53484797]]

 ..., 
 [[ 0.48978743]
  [ 0.45976391]
  [ 0.51348162]
  ..., 
  [ 0.49710327]
  [ 0.49158219]
  [ 0.4706462 ]]

 [[ 0.47837228]
  [ 0.49418497]
  [ 0.55067003]
  ..., 
  [ 0.45194578]
  [ 0.48165801]
  [ 0.51087308]]

 [[ 0.44744301]
  [ 0.51068068]
  [ 0.51481211]
  ..., 
  [ 0.47583234]
  [ 0.48927146]
  [ 0.48255053]]]
Epoch: 0001 D loss: 1.44 G loss: 0.3941
Epoch: 0002 D loss: 1.263 G loss: 0.4965
Epoch: 0003 D loss: 1.081 G loss: 0.6279
Epoch: 0004 D loss: 1.031 G loss: 0.6702
[[[ 0.35424039]
  [ 0.31020936]
  [ 0.30194232]
  ..., 
  [ 0.3548359 ]
  [ 0.36780417]
  [ 0.38071251]]

 [[ 0.39522159]
  [ 0.33822832]
  [ 0.32326436]

Epoch: 0045 D loss: 1.006 G loss: 0.6931
Epoch: 0046 D loss: 1.006 G loss: 0.6931
Epoch: 0047 D loss: 1.006 G loss: 0.6931
Epoch: 0048 D loss: 1.006 G loss: 0.6931
Epoch: 0049 D loss: 1.006 G loss: 0.6931
[[[ 0.19722684]
  [ 0.20850615]
  [ 0.15918066]
  ..., 
  [ 0.1862047 ]
  [ 0.18465216]
  [ 0.2071217 ]]

 [[ 0.24355407]
  [ 0.14970194]
  [ 0.17639345]
  ..., 
  [ 0.18182568]
  [ 0.16113991]
  [ 0.154966  ]]

 [[ 0.20885535]
  [ 0.15685427]
  [ 0.15682413]
  ..., 
  [ 0.18136504]
  [ 0.17704499]
  [ 0.17404588]]

 ..., 
 [[ 0.21379161]
  [ 0.17108393]
  [ 0.16443597]
  ..., 
  [ 0.17261712]
  [ 0.17843688]
  [ 0.20175745]]

 [[ 0.19300117]
  [ 0.1638173 ]
  [ 0.16793428]
  ..., 
  [ 0.16832969]
  [ 0.17805161]
  [ 0.16617063]]

 [[ 0.18509316]
  [ 0.17979869]
  [ 0.18295169]
  ..., 
  [ 0.16795711]
  [ 0.16687587]
  [ 0.16655789]]]
Epoch: 0050 D loss: 1.006 G loss: 0.6931
Epoch: 0051 D loss: 1.006 G loss: 0.6931
Epoch: 0052 D loss: 1.006 G loss: 0.6931
Epoch: 0053 D loss: 1.006 G l

Epoch: 0095 D loss: 1.006 G loss: 0.6931
Epoch: 0096 D loss: 1.006 G loss: 0.6931
Epoch: 0097 D loss: 1.006 G loss: 0.6931
Epoch: 0098 D loss: 1.006 G loss: 0.6931
Epoch: 0099 D loss: 1.006 G loss: 0.6931
[[[ 0.20804535]
  [ 0.17297164]
  [ 0.15392752]
  ..., 
  [ 0.17026415]
  [ 0.17332257]
  [ 0.18866529]]

 [[ 0.19619022]
  [ 0.15425223]
  [ 0.13986808]
  ..., 
  [ 0.17989236]
  [ 0.17196597]
  [ 0.15322587]]

 [[ 0.19701271]
  [ 0.12954378]
  [ 0.126633  ]
  ..., 
  [ 0.15236983]
  [ 0.18060932]
  [ 0.17981839]]

 ..., 
 [[ 0.16359067]
  [ 0.13307814]
  [ 0.16543385]
  ..., 
  [ 0.16721386]
  [ 0.1511941 ]
  [ 0.15485953]]

 [[ 0.15056948]
  [ 0.16632172]
  [ 0.15476438]
  ..., 
  [ 0.14784476]
  [ 0.15682237]
  [ 0.15728603]]

 [[ 0.18500468]
  [ 0.18414123]
  [ 0.16965947]
  ..., 
  [ 0.15325461]
  [ 0.18519267]
  [ 0.14427045]]]
Epoch: 0100 D loss: 1.006 G loss: 0.6931
Epoch: 0101 D loss: 1.006 G loss: 0.6931
Epoch: 0102 D loss: 1.006 G loss: 0.6931
Epoch: 0103 D loss: 1.006 G l

Epoch: 0145 D loss: 1.006 G loss: 0.6931
Epoch: 0146 D loss: 1.006 G loss: 0.6931
Epoch: 0147 D loss: 1.006 G loss: 0.6931
Epoch: 0148 D loss: 1.006 G loss: 0.6931
Epoch: 0149 D loss: 1.006 G loss: 0.6931
[[[ 0.17960404]
  [ 0.14342843]
  [ 0.16757827]
  ..., 
  [ 0.15339273]
  [ 0.16918379]
  [ 0.17370631]]

 [[ 0.17124385]
  [ 0.14815253]
  [ 0.13760306]
  ..., 
  [ 0.15862918]
  [ 0.12559789]
  [ 0.13497961]]

 [[ 0.16839416]
  [ 0.1514755 ]
  [ 0.11681377]
  ..., 
  [ 0.14855811]
  [ 0.14675014]
  [ 0.14945716]]

 ..., 
 [[ 0.1608468 ]
  [ 0.13913368]
  [ 0.16058046]
  ..., 
  [ 0.13952322]
  [ 0.15129477]
  [ 0.14697064]]

 [[ 0.15617761]
  [ 0.15448934]
  [ 0.15778883]
  ..., 
  [ 0.15207437]
  [ 0.15234561]
  [ 0.16059706]]

 [[ 0.15748383]
  [ 0.17084788]
  [ 0.14727075]
  ..., 
  [ 0.13257   ]
  [ 0.15408672]
  [ 0.15402487]]]
Epoch: 0150 D loss: 1.006 G loss: 0.6931
Epoch: 0151 D loss: 1.006 G loss: 0.6931
Epoch: 0152 D loss: 1.006 G loss: 0.6931
Epoch: 0153 D loss: 1.006 G l

Epoch: 0195 D loss: 1.006 G loss: 0.6931
Epoch: 0196 D loss: 1.006 G loss: 0.6931
Epoch: 0197 D loss: 1.006 G loss: 0.6931
Epoch: 0198 D loss: 1.006 G loss: 0.6931
Epoch: 0199 D loss: 1.006 G loss: 0.6931
[[[ 0.08272152]
  [ 0.07502773]
  [ 0.07645705]
  ..., 
  [ 0.08572259]
  [ 0.06584109]
  [ 0.08698213]]

 [[ 0.09502525]
  [ 0.06427263]
  [ 0.06724639]
  ..., 
  [ 0.06499245]
  [ 0.07074542]
  [ 0.05773483]]

 [[ 0.09277827]
  [ 0.05132316]
  [ 0.05187232]
  ..., 
  [ 0.08495368]
  [ 0.07482457]
  [ 0.07064291]]

 ..., 
 [[ 0.0601347 ]
  [ 0.05875774]
  [ 0.0582138 ]
  ..., 
  [ 0.07186366]
  [ 0.07366708]
  [ 0.06584832]]

 [[ 0.07625166]
  [ 0.0679091 ]
  [ 0.06955551]
  ..., 
  [ 0.06587722]
  [ 0.06820194]
  [ 0.09103728]]

 [[ 0.07481421]
  [ 0.08859104]
  [ 0.07602087]
  ..., 
  [ 0.05958297]
  [ 0.07181186]
  [ 0.0640823 ]]]
Epoch: 0200 D loss: 1.006 G loss: 0.6931
Epoch: 0201 D loss: 1.006 G loss: 0.6931
Epoch: 0202 D loss: 1.006 G loss: 0.6931
Epoch: 0203 D loss: 1.006 G l

...

Epoch: 5160 D loss: 1.006 G loss: 0.6931
Epoch: 5161 D loss: 1.006 G loss: 0.6931
Epoch: 5162 D loss: 1.006 G loss: 0.6931
Epoch: 5163 D loss: 1.006 G loss: 0.6931
Epoch: 5164 D loss: 1.006 G loss: 0.6931
[[[ 0.00052055]
  [ 0.00038971]
  [ 0.00038578]
  ..., 
  [ 0.00030563]
  [ 0.00037184]
  [ 0.00040089]]

 [[ 0.00051088]
  [ 0.00024175]
  [ 0.00023501]
  ..., 
  [ 0.00026769]
  [ 0.00022819]
  [ 0.00019719]]

 [[ 0.00041449]
  [ 0.0002391 ]
  [ 0.00016783]
  ..., 
  [ 0.00021086]
  [ 0.00023094]
  [ 0.00021291]]

 ..., 
 [[ 0.00035983]
  [ 0.00020578]
  [ 0.00024105]
  ..., 
  [ 0.00019941]
  [ 0.00024575]
  [ 0.00028358]]

 [[ 0.00029302]
  [ 0.00020335]
  [ 0.00023854]
  ..., 
  [ 0.00027591]
  [ 0.00021332]
  [ 0.00024552]]

 [[ 0.00031904]
  [ 0.00022049]
  [ 0.00021155]
  ..., 
  [ 0.00030632]
  [ 0.00021181]
  [ 0.00020955]]]
Epoch: 5165 D loss: 1.006 G loss: 0.6931
Epoch: 5166 D loss: 1.006 G loss: 0.6931
Epoch: 5167 D loss: 1.006 G loss: 0.6931
Epoch: 5168 D loss: 1.006 G l

Epoch: 5206 D loss: 1.006 G loss: 0.6931
Epoch: 5207 D loss: 1.006 G loss: 0.6931
Epoch: 5208 D loss: 1.006 G loss: 0.6931
Epoch: 5209 D loss: 1.006 G loss: 0.6931
[[[ 0.00777846]
  [ 0.00576385]
  [ 0.0068755 ]
  ..., 
  [ 0.00607146]
  [ 0.00657455]
  [ 0.00645127]]

 [[ 0.00643674]
  [ 0.00514439]
  [ 0.00526353]
  ..., 
  [ 0.00541848]
  [ 0.00515155]
  [ 0.00466174]]

 [[ 0.00681624]
  [ 0.00532959]
  [ 0.00435774]
  ..., 
  [ 0.00495809]
  [ 0.00516374]
  [ 0.00503318]]

 ..., 
 [[ 0.00641329]
  [ 0.00428434]
  [ 0.00493695]
  ..., 
  [ 0.00511645]
  [ 0.00536834]
  [ 0.00489568]]

 [[ 0.00571035]
  [ 0.00449925]
  [ 0.00537093]
  ..., 
  [ 0.00541112]
  [ 0.00454592]
  [ 0.00472086]]

 [[ 0.00749979]
  [ 0.00567041]
  [ 0.0055343 ]
  ..., 
  [ 0.00526212]
  [ 0.00546168]
  [ 0.00519913]]]
Epoch: 5210 D loss: 1.006 G loss: 0.6931
Epoch: 5211 D loss: 1.006 G loss: 0.6931
Epoch: 5212 D loss: 1.006 G loss: 0.6931
Epoch: 5213 D loss: 1.006 G loss: 0.6931
Epoch: 5214 D loss: 1.006 G l

Epoch: 5250 D loss: 1.006 G loss: 0.6931
Epoch: 5251 D loss: 1.006 G loss: 0.6931
Epoch: 5252 D loss: 1.006 G loss: 0.6931
Epoch: 5253 D loss: 1.006 G loss: 0.6931
Epoch: 5254 D loss: 1.006 G loss: 0.6931
[[[ 0.0006585 ]
  [ 0.00061602]
  [ 0.00044714]
  ..., 
  [ 0.00039131]
  [ 0.00051062]
  [ 0.00056463]]

 [[ 0.0005527 ]
  [ 0.0003698 ]
  [ 0.00035457]
  ..., 
  [ 0.00033   ]
  [ 0.00032407]
  [ 0.00039889]]

 [[ 0.00052316]
  [ 0.00030555]
  [ 0.00029493]
  ..., 
  [ 0.00031235]
  [ 0.00028699]
  [ 0.00037422]]

 ..., 
 [[ 0.00048961]
  [ 0.00041447]
  [ 0.00037607]
  ..., 
  [ 0.00031308]
  [ 0.00038142]
  [ 0.00037224]]

 [[ 0.00049522]
  [ 0.00035907]
  [ 0.00030059]
  ..., 
  [ 0.0002842 ]
  [ 0.00035785]
  [ 0.00032648]]

 [[ 0.00056404]
  [ 0.00038176]
  [ 0.00039076]
  ..., 
  [ 0.00040406]
  [ 0.0003644 ]
  [ 0.00033617]]]
Epoch: 5255 D loss: 1.006 G loss: 0.6931
Epoch: 5256 D loss: 1.006 G loss: 0.6931
Epoch: 5257 D loss: 1.006 G loss: 0.6931
Epoch: 5258 D loss: 1.006 G l

[[[ 0.00264383]
  [ 0.00262945]
  [ 0.00176917]
  ..., 
  [ 0.00208705]
  [ 0.0021384 ]
  [ 0.00256561]]

 [[ 0.00213643]
  [ 0.00174054]
  [ 0.00135991]
  ..., 
  [ 0.00189473]
  [ 0.00160632]
  [ 0.00166219]]

 [[ 0.0020577 ]
  [ 0.00149288]
  [ 0.00141297]
  ..., 
  [ 0.0013009 ]
  [ 0.00153638]
  [ 0.00143703]]

 ..., 
 [[ 0.00223403]
  [ 0.00144067]
  [ 0.00153862]
  ..., 
  [ 0.00147118]
  [ 0.00149306]
  [ 0.00143843]]

 [[ 0.00167059]
  [ 0.00149822]
  [ 0.00127888]
  ..., 
  [ 0.00143896]
  [ 0.00128297]
  [ 0.0016171 ]]

 [[ 0.0023366 ]
  [ 0.00165259]
  [ 0.00146791]
  ..., 
  [ 0.00150735]
  [ 0.00157942]
  [ 0.00124851]]]
Epoch: 5300 D loss: 1.006 G loss: 0.6931
Epoch: 5301 D loss: 1.006 G loss: 0.6931
Epoch: 5302 D loss: 1.006 G loss: 0.6931
Epoch: 5303 D loss: 1.006 G loss: 0.6931
Epoch: 5304 D loss: 1.006 G loss: 0.6931
[[[ 0.00650764]
  [ 0.00573014]
  [ 0.00529155]
  ..., 
  [ 0.00495599]
  [ 0.00424482]
  [ 0.00566173]]

 [[ 0.00604751]
  [ 0.00353341]
  [ 0.00360807

KeyboardInterrupt: 